In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install bitsandbytes accelerate xformers einops
!pip install datasets

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = transformers.BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
model_id,
trust_remote_code=True,
quantization_config=bnb_config,
device_map='auto',
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
model_id,
)

# text = "[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen! [INST] Do you have mayonnaise recipes? [/INST]"
# encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)
# model_inputs = encodeds.to('cuda')

# generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True)
# decoded = tokenizer.batch_decode(generated_ids)
# print(decoded[0])

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/mistral

Mounted at /content/drive
/content/drive/MyDrive/mistral


In [3]:
from datasets import load_dataset
raw_datasets = load_dataset("orkg/SciQA")
print(raw_datasets)

Generating train split:   0%|          | 0/1795 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/513 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 1795
    })
    validation: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 257
    })
    test: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 513
    })
})


In [4]:
import json
import random

def save_json(filename,data):
    with open(filename, "w", encoding="utf-8") as json_file:
        print(json.dumps(data), file=json_file)


def load_json(file__name):
    try:
        data_file = open(file__name, "r", encoding='utf-8')
        file_data = json.loads(data_file.read())
        data_file.close()
        return file_data
    except FileNotFoundError:
        return None


def clean(st):
    st = st.replace("\n", " ")
    st = st.replace("?", " ?")
    st = st.replace("{", " { ")
    st = st.replace("}", " } ")
    st = st.replace("\\'", "'")

    while "  " in st:
        st = st.replace("  ", " ")
    return st


def get_key(q):
    t0 = q.get('template_id')
    if t0 is None:
        t0 = "None"
    t = str(q.get("number_of_patterns")) + "-" + t0
    return t


def get_random(n_):
    train = raw_datasets.get("train")
    sample = random.sample(list(train), n_)
    sample_list = []
    for q in sample:
        t = get_key(q)
        query = clean(q["query"]["sparql"])
        question = q["question"]["string"]
        sample_list.append([query, question, t])
    return sample_list


def prepare_queries(n_):
    data = raw_datasets.get("test")
    queries = []
    suggestions = []
    for q in data:
        t = get_key(q)
        question = q["question"]["string"]

        if n_ == 0:
            # queries.append("[INST]translate to a SPARQL query for ORKG the following English text: '"+question+"'. Return only the SPARQL query, without any explanation.[/INST]")
            queries.append("<s>[INST]translate to a SPARQL query the following English text: '"+question+"'. Return just the SPARQL query, without any explanation or added courtesy.[/INST]")
            # queries.append("<s>[INST]translate to a SPARQL query for ORKG the following English text: '"+question+"'. Return only the SPARQL query, without any explain.[/INST]")
            suggestions.append(t)
        else:

            suggestion = get_random(n_)
            suggestions.append([[x[2] for x in suggestion], t])
            # print(suggestion, t)
            # break

            if suggestion is None or len(suggestion) == 0:
                print("Error with key", t)
                # queries.append("[INST]translate to a SPARQL query for ORKG the following English text: '"+question+"'. Return only the SPARQL query, without any explanation.[/INST]")
                queries.append("<s>[INST]translate to a SPARQL query the following English text: '"+question+"'. Return just the SPARQL query, without any explanation or added courtesy.[/INST]")
                # queries.append("<s>[INST]translate to a SPARQL query for ORKG the following English text: '"+question+"'. Return only the SPARQL query, without any explain.[/INST]")
            else:
                final_q = ""
                for i_, k in enumerate(suggestion):

                    # works better with fine-tuned gpt2?
                    # final_q += "<|endoftext|>" + k[1] + " "
                    # final_q += k[0]

                    # works better with dolly
                    # final_q += "\n input (English text): " + k[1]
                    # final_q += "\n output (Sparql query): " + k[0]

                    # works better with mistral?
                    final_q += "<s>[INST]input (English text): " + k[1]
                    final_q += "\n output (Sparql query): [/INST]" + k[0] + "</s>"

                # works better with mistral?
                # final_q += "\n with these as example what is the SPARQ query on ORKG for: '" + question + "'. Return only the SPARQL query, without any explanation.[/INST]"
                # final_q += "\n with these as example translate to a SPARQL query for ORKG the following English text: '" + question + "'. Return only the SPARQL query, without any explain.[/INST]"
                final_q += "<s>[INST]input (English text): " + question
                final_q += "\n output (Sparql query): [/INST]"

                # works better with gpt2?
                # final_q += "<|endoftext|>" + question

                # works better with dolly
                # final_q += "\n input (English text): " + question
                # final_q += "\n output (Sparql query): "

                queries.append(final_q)

    return queries, suggestions


def main(shots=0, filename="mistral"):
    data = load_json(filename + "_" + str(shots) + "_shots.json")
    # print(data)
    if data is None:
        query_list, suggestions = prepare_queries(shots)
        gs = []
        lens = []
    else:
        query_list = data["questions"]
        suggestions = data["suggestions"]
        gs = data["generated_sparql"]
        lens = data["prompt_len"]

    print(len(query_list))

    q_list = query_list
    sparql = [clean(x["query"]["sparql"]) for x in raw_datasets.get("test")]

    for question in q_list[len(gs):]:
        print(len(gs))
        encodeds = tokenizer(question, return_tensors="pt", add_special_tokens=False)
        model_inputs = encodeds.to('cuda')

        generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
        decoded = tokenizer.batch_decode(generated_ids)
        print(decoded[0])

        gs.append(decoded[0])
        result = {"questions": query_list, "sparql": sparql, "generated_sparql": gs, "prompt_len": lens, "suggestions": suggestions}
        save_json(filename + "_" + str(shots) + "_shots.json", result)



In [5]:
main(7)

513
141


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]input (English text): Which model has achieved the highest Accuracy (%) score on the  Jacquard dataset benchmark dataset?
 output (Sparql query): [/INST]SELECT DISTINCT ?model ?model_lbl WHERE { ?metric a orkgc:Metric; rdfs:label ?metric_lbl. FILTER (str( ?metric_lbl) = "Accuracy (%)") { SELECT ?model ?model_lbl WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = " Jacquard dataset") ?benchmark orkgp:HAS_DATASET ?dataset; orkgp:HAS_EVALUATION ?eval. ?eval orkgp:HAS_VALUE ?value; orkgp:HAS_METRIC ?metric. ?cont orkgp:HAS_BENCHMARK ?benchmark; orkgp:HAS_MODEL ?model. ?model rdfs:label ?model_lbl. } ORDER BY DESC( ?value) LIMIT 1 } } </s><s> [INST]input (English text): What is the highest benchmark result achieved on the Open Entity dataset, including the metric and its value?
 output (Sparql query): [/INST]SELECT DISTINCT ?metric ?metric_lbl (MAX( ?value) AS ?score) WHERE { { SELECT ?metric ?metric_lbl ?value WHERE { ?dataset a orkgc:Dataset;

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]input (English text): Provide a list of research paper titles and IDs that have benchmarked models on the Atari 2600 Star Gunner dataset?
 output (Sparql query): [/INST]SELECT DISTINCT ?paper ?paper_lbl WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "Atari 2600 Star Gunner") ?benchmark orkgp:HAS_DATASET ?dataset. ?cont orkgp:HAS_BENCHMARK ?benchmark. ?paper orkgp:P31 ?cont; rdfs:label ?paper_lbl. } </s><s> [INST]input (English text): Where can I find code references in papers that have used the GRU model for benchmarking purposes?
 output (Sparql query): [/INST]SELECT DISTINCT ?code WHERE { ?model a orkgc:Model; rdfs:label ?model_lbl. FILTER (str( ?model_lbl) = "GRU") ?benchmark orkgp:HAS_DATASET ?dataset. ?cont orkgp:HAS_BENCHMARK ?benchmark. ?cont orkgp:HAS_MODEL ?model; orkgp:HAS_SOURCE_CODE ?code. } </s><s> [INST]input (English text): Can you provide links to code used in papers that benchmark the Random Forest model?
 output (Spar

...

<s> [INST]input (English text): What models are being evaluated on the CORLL dataset?
 output (Sparql query): [/INST]SELECT DISTINCT ?model ?model_lbl WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "CORLL") ?benchmark orkgp:HAS_DATASET ?dataset; orkgp:HAS_EVALUATION ?eval. ?paper orkgp:HAS_BENCHMARK ?benchmark. OPTIONAL { ?paper orkgp:HAS_MODEL ?model. ?model rdfs:label ?model_lbl. } } </s><s> [INST]input (English text): Give me a list of research papers along with their titles and IDs, that have performed benchmarks on the TACRED dataset?
 output (Sparql query): [/INST]SELECT DISTINCT ?paper ?paper_lbl WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "TACRED") ?benchmark orkgp:HAS_DATASET ?dataset. ?cont orkgp:HAS_BENCHMARK ?benchmark. ?paper orkgp:P31 ?cont; rdfs:label ?paper_lbl. } </s><s> [INST]input (English text): Can you list benchmarked problems in the area of Computer Sciences?
 output (Sparql quer

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]input (English text): Give me a list of research papers along with their titles and IDs, that have performed benchmarks on the ANLI test dataset?
 output (Sparql query): [/INST]SELECT DISTINCT ?paper ?paper_lbl WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "ANLI test") ?benchmark orkgp:HAS_DATASET ?dataset. ?cont orkgp:HAS_BENCHMARK ?benchmark. ?paper orkgp:P31 ?cont; rdfs:label ?paper_lbl. } </s><s> [INST]input (English text): Indicate the model that performed best in terms of Accuracy metric on the MNIST benchmark dataset?
 output (Sparql query): [/INST]SELECT DISTINCT ?model ?model_lbl WHERE { ?metric a orkgc:Metric; rdfs:label ?metric_lbl. FILTER (str( ?metric_lbl) = "Accuracy") { SELECT ?model ?model_lbl WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "MNIST") ?benchmark orkgp:HAS_DATASET ?dataset; orkgp:HAS_EVALUATION ?eval. ?eval orkgp:HAS_VALUE ?value; orkgp:HAS_METRIC ?metric. ?cont ork

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]input (English text): List the code links in papers that use the AttentionOCR_Inception-resnet-v2_Location model in any benchmark?
 output (Sparql query): [/INST]SELECT DISTINCT ?code WHERE { ?model a orkgc:Model; rdfs:label ?model_lbl. FILTER (str( ?model_lbl) = "AttentionOCR_Inception-resnet-v2_Location") ?benchmark orkgp:HAS_DATASET ?dataset. ?cont orkgp:HAS_BENCHMARK ?benchmark. ?cont orkgp:HAS_MODEL ?model; orkgp:HAS_SOURCE_CODE ?code. } </s><s> [INST]input (English text): What are the titles and IDs of research papers that include a benchmark for the BioASQ dataset?
 output (Sparql query): [/INST]SELECT DISTINCT ?paper ?paper_lbl WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "BioASQ") ?benchmark orkgp:HAS_DATASET ?dataset. ?cont orkgp:HAS_BENCHMARK ?benchmark. ?paper orkgp:P31 ?cont; rdfs:label ?paper_lbl. } </s><s> [INST]input (English text): What is the top benchmark score and its metric on the MLDoc Zero-Shot English-to-Japan